In [0]:
%run "../includes/configuration"

In [0]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
race_results_df = race_results_df.filter("race_year = 2020")

In [0]:
from pyspark.sql.functions import when, col, count, sum

In [0]:
race_aggregated_df = race_results_df \
    .groupBy("race_year", "driver_name" , "team") \
    .agg(sum("points").alias("Points"), 
         count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(race_aggregated_df)

In [0]:


from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

window = Window.partitionBy("race_year").orderBy(desc("Points"), desc("wins"))
final_df = race_aggregated_df.withColumn("rank", rank().over(window))

In [0]:
display(final_df.filter("race_year = 2020"))

In [0]:
final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standings")